# Homework 3 - Interactive Viz

In [ ]:
import folium
import requests as rq
from bs4 import BeautifulSoup as bfs
import numpy as np
import pandas as pd
import collections
import os
%matplotlib inline
import matplotlib.pyplot as plt
from dateutil import relativedelta
import scipy.stats as stats
import math

In [ ]:
state_geo = r'ch-cantons.topojson.json'

#Let Folium determine the scale
map = folium.Map(location=[46.8, 7.1], zoom_start=8)
#map.choropleth(geo_path=state_geo, fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2)
#map.save('us_states.html')
map

In [ ]:
snsf_data = pd.read_csv('Data/P3_GrantExport.csv', sep=';')
snsf_data.head()

In [ ]:
snsf_data.dtypes

In [ ]:
snsf_data.columns = snsf_data.columns.str.replace(snsf_data.columns[0], 'Project Number')
snsf_data.head()

In [ ]:
snsf_data['Project Number'].is_unique

In [ ]:
snsf_data.set_index('Project Number')

In [ ]:
snsf_data['University'].value_counts(dropna=False)
#snsf_data['University'].value_uniques()